<a href="https://colab.research.google.com/github/jrcristofani/Projeto-Imersao-Alura-Google/blob/main/CrewAI_Equipe_Exegetica_com_Pesquisa_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ PASSO 1 – Instalar pacotes necessários

In [ ]:
# ✅ PASSO 1 – Instalação dos pacotes
!pip install crewai crewai-tools
!pip install "httpx<1.0.0,>=0.28.1" "tokenizers<0.22,>=0.21"
!pip install json-repair chromadb

  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.9.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.2
    Uninstalling httpx-0.27.2:
      Successfully uninstalled http

✅ PASSO 2 – Importar libs e configurar chaves com API secrets

In [ ]:
# ✅ PASSO 2 – Configuração com API secrets (do Colab)
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = userdata.get("SERPER_API_KEY")

# Confirmação visual
print("✅ Chaves carregadas com sucesso!")

✅ Chaves carregadas com sucesso!


✅ PASSO 3 – Criar ferramentas para pesquisa e vocabulário

In [ ]:
# ✅ PASSO 3 – Ferramentas para pesquisa, scraping e léxico simulado

from crewai_tools import SerperDevTool
from langchain.tools import BaseTool
from typing import Optional, Type
from pydantic import BaseModel, Field

# 🔍 Ferramenta de busca Serper
search_tool = SerperDevTool()

# ✅ Ferramenta Lexical Simulada (100% compatível com CrewAI)
class LexicoInput(BaseModel):
    termo: str = Field(..., description="Termo bíblico em hebraico ou grego")

class LexicoTool(BaseTool):
    name: str = "consultar_lexico"
    description: str = "Consulta termos em um léxico bíblico simulado (hebraico ou grego)"
    args_schema: Type[BaseModel] = LexicoInput

    def _run(self, termo: str) -> str:
        termos_exemplo = {
            "pecado": "ḥaṭṭāʼṯ (חַטָּאת) = falha moral, quebra da aliança com Deus. Cf. HALOT.",
            "justiça": "ṣĕḏāqâ (צְדָקָה) = justiça conforme a aliança divina. Cf. TLOT.",
            "sabedoria": "ḥoḵmāh (חָכְמָה) = sabedoria prática e ética. Cf. TWOT.",
        }
        return termos_exemplo.get(termo.lower(), "❌ Termo não encontrado no léxico local.")

    def _arun(self, *args, **kwargs):
        raise NotImplementedError("Execução assíncrona não suportada.")

# ✅ Instância do LexicoTool
lexico_tool = LexicoTool()

# Confirmação
print("✅ Ferramentas definidas com sucesso!")

✅ Ferramentas definidas com sucesso!


✅ PASSO 4 – Input do usuário

In [ ]:
from IPython.display import display, HTML

display(HTML("<h3>📥 Informe a referência bíblica para iniciar a exegese (ex: Amós 1.3–5)</h3>"))
referencia_biblica = input("📖 Digite a referência bíblica desejada: ").strip()

if not referencia_biblica:
    raise ValueError("❌ É necessário informar uma referência bíblica para continuar.")

📖 Digite a referência bíblica desejada: Mateus 6.9-15


✅ PASSO 5 – Definição refinada dos agentes exegéticos

In [ ]:
# ✅ PASSO 5 – Definição dos Agentes Exegéticos

from crewai import Agent

# ✅ Agente: Delimitador
agente_delimitador = Agent(
    role="Especialista em Delimitação de Perícopes",
    goal="Delimitar a perícope com base na comparação com as seções imediatamente anterior e posterior",
    backstory=(
        "Você é um especialista em análise textual bíblica, com domínio em crítica literária e metodologia de delimitação de perícope. "
        "Você deve utilizar princípios de coesão textual, unidade temática e literária, além de critérios clássicos de mudança de tempo, lugar ou personagem. "
        "A perícope deve ser analisada em contraste com os versículos imediatamente anteriores e posteriores."
    ),
    tools=[search_tool],
    verbose=True,
    memory=True
)

# ✅ Agente: Gênero Literário
agente_genero = Agent(
    role="Especialista em Formas e Gêneros Literários",
    goal="Classificar a Forma e o Gênero da perícope, explicando sua estrutura típica e Sitz-im-Leben",
    backstory=(
        "Você é um especialista na crítica das formas (Formgeschichte), seguindo a tradição germânica. "
        "Sua tarefa é distinguir claramente entre Forma Literária (tipo de literatura: profética, sapiencial, apocalíptica, etc.) e Gênero Literário (ex: oráculo de juízo, lamento, visão simbólica, etc.). "
        "Além disso, deve identificar o Sitz-im-Leben, o ambiente vivencial em que esse gênero surgiu e era utilizado na comunidade de fé."
    ),
    tools=[search_tool],
    verbose=True,
    memory=True
)

# ✅ Agente: Análise Semântica (sem ferramenta customizada!)
agente_semantico = Agent(
    role="Especialista em Semântica Bíblica",
    goal="Realizar análise lexical rigorosa agrupando termos por eixos semânticos com base em BHS e NA28.",
    backstory=(
        "Você é um expert em hebraico e grego bíblico, com domínio de léxicos como HALOT, TWOT, TLOT, TDNT e BDB. "
        "Seu trabalho é identificar os termos mais relevantes da perícope e organizá-los em campos semânticos, explicando seus significados e implicações teológicas. "
        "Mesmo sem acesso direto aos dicionários, você deve simular uma análise fundamentada com linguagem técnica e acessível, mencionando a fonte apropriada (ex: Cf. HALOT)."
    ),
    tools=[search_tool],  # ❌ Removido lexico_tool
    verbose=True,
    memory=True
)

# ✅ Agente: Contexto Literário e Sócio-histórico
agente_contexto = Agent(
    role="Especialista em Contexto Bíblico",
    goal="Fornecer análise literária e sócio-histórica completa da perícope",
    backstory=(
        "Você é um estudioso experiente em crítica literária e história social do mundo bíblico. "
        "Sua tarefa é estabelecer como o texto se relaciona com as perícopes adjacentes, a seção maior, o livro como um todo e outros textos bíblicos (intertextualidade). "
        "Também deve analisar a sociedade da época — sua estrutura, conflitos, opressões e valores históricos — conforme refletidos no texto."
    ),
    tools=[search_tool],
    verbose=True,
    memory=True
)

# ✅ Agente: Redator Final
agente_redator = Agent(
    role="Teólogo, Professor, Autor",
    goal="Reunir todas as análises anteriores em um texto fluido, rigoroso e pastoral",
    backstory=(
        "Você é um excelente Teólogo, Autor e Professor. Escreve com profundidade e clareza, aliando rigor acadêmico a um estilo pastoral. "
        "Tem habilidade de tornar conceitos complexos em algo compreensível e inspirador, guiando leitores à reflexão, transformação e aplicação prática dos princípios bíblicos."
    ),
    tools=[],
    verbose=True,
    memory=True
)

✅ 📘 PASSO 6 – Task: Redação Final Pastoral e Didática

In [ ]:
from crewai import Task

# 📌 Tarefa 1 – Delimitação da Perícope
task_delimitacao = Task(
    description=f"""
Delimite a perícope fornecida analisando os versículos imediatamente anteriores e posteriores.
Considere os seguintes critérios técnicos:

1. Mudança de tempo verbal
2. Mudança de espaço, personagem ou pessoa gramatical
3. Coesão e coerência literária
4. Unidade temática

📘 Utilize linguagem técnica, articule a análise com os versículos anterior e posterior.
🔖 Transcreva a perícope delimitada como citação direta no final.

Texto-base: {referencia_biblica}
""",
    expected_output="Texto com pelo menos 600 palavras explicando a delimitação, justificativas técnicas e citação da perícope.",
    agent=agente_delimitador
)

# 📌 Tarefa 2 – Forma e Gênero Literário
task_genero = Task(
    description=f"""
Analise a forma e o gênero literário da perícope delimitada:

1. Classifique a Forma Literária (ex: profética, narrativa, apocalíptica etc.)
2. Identifique o Gênero Literário dentro da forma (ex: oráculo, lamento, visão, etc.)
3. Explique o Sitz-im-Leben (ambiente vital em que o gênero surgiu e era usado)

📘 Use terminologia da crítica das formas (Gunkel, Bultmann, Berger).
🔍 Explique a função litúrgica ou social do gênero na comunidade de fé.

Texto-base: {referencia_biblica}
""",
    expected_output="Texto com aproximadamente 800 palavras estruturado em 3 seções com citações bíblicas e referências técnicas.",
    agent=agente_genero
)

# 📌 Tarefa 3 – Análise Semântica Detalhada
task_semantica = Task(
    description=f"""
Realize uma análise semântica rigorosa da perícope. Siga os passos:

1. Liste os principais termos teológicos e relevantes do texto
2. Agrupe-os em Horizontes (ou Eixos) Semânticos com base na relação de sentido
3. Analise os termos com seus equivalentes hebraicos ou gregos (ex: mishpat (מִשְׁפָּט), logos (λόγος)), citando léxicos como HALOT, TWOT, TLOT, TDNT, BDAG

📘 Cada termo deve estar em parênteses com o original hebraico/greek, e vir seguido de definição técnica e teológica.

📝 Exemplo:
Horizonte Semântico: Justiça
- mishpat (מִשְׁפָּט): julgamento legal. HALOT.
- tsedeq (צֶדֶק): justiça ética e relacional. TWOT.

Texto-base: {referencia_biblica}
""",
    expected_output="Texto com cerca de 800 palavras, estruturado por horizontes semânticos com análise lexical teológica.",
    agent=agente_semantico
)

# 📌 Tarefa 4 – Contexto Literário e Sócio-Histórico
task_contexto = Task(
    description=f"""
Analise o contexto da perícope a partir de dois eixos principais:

I. Literário:
- Relação com perícopes anterior e posterior
- Inserção na seção maior do livro
- Papel na macroestrutura do livro
- Intertextualidade bíblica (ecos, alusões, citações)

II. Sócio-Histórico:
- Situação social, política e religiosa implícita no texto
- Conflitos sociais, estruturas de poder e contexto cultural da época
- Conexão com a sociedade do Antigo/Novo Oriente ou do Império Romano (conforme o caso)

📘 Use dados de crítica literária e histórica confiáveis. Mantenha o foco nas evidências textuais.

Texto-base: {referencia_biblica}
""",
    expected_output="Texto com cerca de 800 palavras, dividido claramente entre contexto literário e sócio-histórico.",
    agent=agente_contexto
)

# 📌 Tarefa 5 – Redação Exegética Acadêmica
task_estruturador = Task(
    description=f"""
Com base nos resultados das análises anteriores (delimitação, forma/gênero, semântica, contexto), redija a exegese final.

📘 Estrutura esperada:
1. Introdução com contextualização
2. Tradução com base na NVI, justificando escolhas
3. Delimitação explicada
4. Forma e Gênero Literário (com Sitz-im-Leben)
5. Análise Semântica (com original hebraico/grego e léxicos)
6. Análise Literária e Sócio-Histórica
7. Conclusão com possíveis aplicações e lacunas

🧠 O texto deve ser acadêmico, fluido, técnico, com transições e citações bíblicas referenciadas.

Texto-base: {referencia_biblica}
""",
    expected_output="Texto com entre 1500 e 2000 palavras, acadêmico, coeso e formatado por seções.",
    agent=agente_redator
)

# 📌 Tarefa 6 – Redação Final Pastoral e Inspiradora
task_redator = Task(
    description=f"""
A partir das análises anteriores, reescreva a exegese de forma **fluida, inspiradora e pastoral**, com linguagem acessível e rica em teologia bíblica aplicada.

🎯 Estrutura sugerida:
1. Introdução instigante
2. Tradução comentada
3. Delimitação (resumida e aplicada)
4. Forma e Gênero (com sentido existencial)
5. Semântica (relacionada à vida cristã)
6. Contexto (com implicações atuais)
7. Conclusão pastoral com aplicações para vida, igreja, liderança

📝 Mantenha citações bíblicas (com referências), clareza e profundidade. Conecte o texto à transformação espiritual e prática cristã.

Texto-base: {referencia_biblica}
""",
    expected_output="Texto com entre 2000 e 3000 palavras. Deve ser pastoral, teológico e motivador, com impacto espiritual.",
    agent=agente_redator
)

✅ 📘 PASSO 7 – Execução com visualização e estrutura sequencial

In [ ]:
# ✅ 📘 PASSO 7 – Execução com visualização e estrutura sequencial
from crewai import Crew, Process

# ✅ Criação da equipe com todos os agentes e tarefas exegéticas
crew = Crew(
    agents=[
        agente_delimitador,
        agente_genero,
        agente_semantico,
        agente_contexto,
        agente_redator  # 🆕 Redator Pastoral e Teológico
    ],
    tasks=[
        task_delimitacao,
        task_genero,
        task_semantica,
        task_contexto,
        task_estruturador,
        task_redator  # 🆕 Nova Tarefa de Redação Final
    ],
    process=Process.sequential,
    max_iterations=5
)

# ✅ Execução da crew
print("\n🚀 Iniciando exegese completa...\n")
resultado = crew.kickoff(inputs={
    "referencia_biblica": referencia_biblica,
})

# ✅ Exibição formatada dos resultados
print("\n📘 RESULTADOS EXEGÉTICOS\n" + "="*70)
for idx, task in enumerate(crew.tasks):
    print(f"\n🔹 [{idx+1}] {task.agent.role}")
    print("-"*60)
    print(task.output)
    print("\n" + "="*70)

# ✅ Consolidação dos resultados para exportação
resultado_str = ""
for idx, task in enumerate(crew.tasks):
    resultado_str += f"## {task.agent.role}\n\n{task.output}\n\n"


🚀 Iniciando exegese completa...

# Agent: Especialista em Delimitação de Perícopes
## Task: 
Delimite a perícope fornecida analisando os versículos imediatamente anteriores e posteriores.
Considere os seguintes critérios técnicos:

1. Mudança de tempo verbal
2. Mudança de espaço, personagem ou pessoa gramatical
3. Coesão e coerência literária
4. Unidade temática

📘 Utilize linguagem técnica, articule a análise com os versículos anterior e posterior.
🔖 Transcreva a perícope delimitada como citação direta no final.

Texto-base: Mateus 6.9-15

 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for SerperDevToolSchema
search_query
  Field required [type=missing, input_value={'description': 'Delimita...0394', 'metadata': {}}}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Se

✅ PASSO 8 – Exportar resultado para arquivos .txt e .md

In [ ]:
from datetime import datetime

# 📅 Nome com timestamp para evitar sobrescrever
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
nome_base = f"exegese_{referencia_biblica.replace('.', '-').replace(':', '-')}_{timestamp}"

# 📝 Exporta para .txt
with open(f"{nome_base}.txt", "w", encoding="utf-8") as txt_file:
    txt_file.write(resultado_str)

# 📄 Exporta para .md
with open(f"{nome_base}.md", "w", encoding="utf-8") as md_file:
    md_file.write(resultado_str)

print("\n✅ Arquivos exportados com sucesso!")
print(f"📄 {nome_base}.txt")
print(f"📄 {nome_base}.md")


✅ Arquivos exportados com sucesso!
📄 exegese_Mateus 6-9-15_20250411_212740.txt
📄 exegese_Mateus 6-9-15_20250411_212740.md
